# Import Libraries

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from art.utils import load_mnist
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.extraction import CopycatCNN

# Load Data

In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

# Target Model

In [4]:
target_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation="relu"),
    Dense(10)
])

In [5]:
target_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [6]:
target_model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=1)

Epoch 1/5
938/938 [==============================] - 5s 5ms/step - loss: 0.2159 - accuracy: 0.9342
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1336 - accuracy: 0.9609
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1118 - accuracy: 0.9674
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1039 - accuracy: 0.9715
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0887 - accuracy: 0.9750


In [20]:
target_classifier = TensorFlowV2Classifier(
    model=target_model, 
    loss_object=CategoricalCrossentropy(from_logits=True), 
    optimizer=Adam(learning_rate=0.01), 
    nb_classes=10,
    input_shape=(28, 28),
    clip_values=(min_, max_)
)

In [21]:
print("Target Model Accuracy:", target_model.evaluate(x_test, y_test, verbose=0)[1])

Target Model Accuracy: 0.965499997138977


# CopycatCNN

In [36]:
copycat_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation="relu"),
    Dense(10)
])

In [37]:
copycat_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [38]:
copycat_classifier = TensorFlowV2Classifier(
    model=copycat_model, 
    loss_object=CategoricalCrossentropy(from_logits=True), 
    optimizer=Adam(learning_rate=0.01),
    nb_classes=10,
    input_shape=(28, 28),
    clip_values=(min_, max_)
)

In [39]:
copycat_attack = CopycatCNN(
    classifier=copycat_classifier, 
    nb_epochs=10,
    nb_stolen=10000
)

In [40]:
copycat_attack.extract(x_test, y_test, thieved_classifier=target_classifier)

art.estimators.classification.tensorflow.TensorFlowV2Classifier(model=<keras.src.engine.sequential.Sequential object at 0x7f38c37e3640>, nb_classes=10, input_shape=(28, 28), loss_object=<keras.src.losses.CategoricalCrossentropy object at 0x7f3834566890>, optimizer=<keras.src.optimizers.adam.Adam object at 0x7f381556ff70>, train_step=None, channels_first=False, clip_values=array([0., 1.], dtype=float32), preprocessing_defences=None, postprocessing_defences=None, preprocessing=StandardisationMeanStdTensorFlow(mean=0.0, std=1.0, apply_fit=True, apply_predict=True))

In [42]:
print("CopycatCNN Model Accuracy:", copycat_classifier._model.evaluate(x_test, y_test, verbose=0)[1])

CopycatCNN Model Accuracy: 0.11299999803304672
